# downloading ALOS-2 aux data

When using ISCE to process ALOS-2 data, the aux data(dem1, dem3, wbd) should be specified. following code is used to prepare the stack folder structure from raw ALOS data and download those aux data into this home folder.

In [ ]:
import os
from pathlib import Path

# raw data of unzipped ALOS-2
data_dir = Path('/data/alos2/data')
# folder for processing the ALOS-2
stack_dir = Path('/data/alos2/isce_process')
# folder contains ALOS-2 data linked from 'data_dir' folder
alos_data_dir = stack_dir / 'alos_data'

files = sorted(data_dir.rglob('*.1__D-F1'))


def ensure_dir(folder):
    if not folder.is_dir():
        folder.mkdir(parents=True)

Let's link the raw ALOS-2 data to the processing folder

In [ ]:
######################  link data ##################
# specify the date to use
date_used = ['210919', '211031', '211212', '220123', '220114']
# date_used = None    # for all available date

for file in files:
    raw_folder = file.parent
    name = file.stem
    date = name.split('-')[-2]
    if date_used is not None:
        if not date in date_used:
            continue

    date_dir = alos_data_dir / date
    ensure_dir(date_dir)

    ln_cmd = f'ln -sf {raw_folder}/* {date_dir}/'
    os.system(ln_cmd)

Specify the aux folders

In [ ]:
######################  aux data ##################
aux_dir = stack_dir / 'aux_data'
ensure_dir(aux_dir)

dem1_dir = aux_dir / 'dem_1_arcsec'
ensure_dir(dem1_dir)
dem3_dir = aux_dir / 'dem_3_arcsec'
ensure_dir(dem3_dir)

wb_dir = aux_dir / 'wbd_1_arcsec'
ensure_dir(wb_dir)

Downloading the aux files

In [ ]:
# %%  download aux

box = '36 40 97 103'  # the extent of dem with order [S, N, W, E]
os.chdir(dem1_dir)
dl_dem1 = f'dem.py -a stitch -b {box} -k -s 1 -c -f -u https://e4ftl01.cr.usgs.gov/MEASURES/SRTMGL1.003/2000.02.11'
os.system(dl_dem1)

os.chdir(dem3_dir)
dl_dem3 = f'dem.py -a stitch -b {box} -k -s 3 -c -f -u https://e4ftl01.cr.usgs.gov/MEASURES/SRTMGL3.003/2000.02.11'
os.system(dl_dem3)

os.chdir(wb_dir)
wb_dem3 = f'wbd.py {box}'
os.system(wb_dem3)